## Determine Main Contributers to Electricity RRP

## Stage 2: Data Processing to produce Dataset of 30 min Average of 5min Generator Data

In [2]:
# Import libraries 
# I've included a handful of libraries here that might be useful
import glob
import os, zipfile, shutil
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import sys
import time
from IPython.display import display # Used to display multiple pandas tables in one cell
from datetime import datetime
from datetime import timedelta
from scipy import stats
from pprint import pprint
%matplotlib inline

## SCADA Data for Generator Information

Source Data is AEMO Dispatch Data from http://www.nemweb.com.au/REPORTS/ARCHIVE/Dispatch_SCADA/. 
One month at a time was analysed for this project.

This file contains actual generation data for each scheduled generation unit, semi-scheduled generation unit, and non-scheduled generating units or non-scheduled generating systems (a non-scheduled generating system comprising non-scheduled generating units). This file also contains actual load for each scheduled load. The actual generation and load data in the file is reported under a column headed 'SCADAVALUE'.

In [ ]:
from_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DISPATCHSCADA'
to_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DispatchScada_5minFiles'
extension = ".zip"

os.chdir(from_dir) # change directory from working dir to dir with files

for item in os.listdir(from_dir): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(to_dir) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [ ]:
from_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DispatchScada_5minFiles'
to_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DispatchScada_5minFilesUnzipped'
extension = ".zip"

os.chdir(from_dir) # change directory from working dir to dir with files

for item in os.listdir(from_dir): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(to_dir) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [ ]:
analysisPath = 'F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchScada_5minFilesUnzipped/'
count = len([f for f in os.listdir(analysisPath) if os.path.isfile(os.path.join(analysisPath, f))])
print(count)

In [ ]:
#Change this if necessary
numfiles = 0

In [ ]:
# Set the path to the data files - change this to where you store your files
analysisPath = 'F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchScada_5minFilesUnzipped/'
destinationPath = 'F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchScada_5minFilesProcessed/'

In [ ]:
# Process files in 1000 blocks
count = 0
df_temp = pd.DataFrame()
for file in os.listdir(analysisPath):
    if (count < 1000):  
        if file.endswith(".CSV"):
            count = count + 1
            if count % 100 == 0:
                print(count)
            input_file = os.path.join(analysisPath, file)
            dest_file = os.path.join(destinationPath, file)
            # Read the csv files into dataframes
            df_input = pd.read_csv(input_file,sep=',',skiprows=1,error_bad_lines=False,warn_bad_lines=False)
            # Move file once processed
            shutil.move(input_file, dest_file)
            df_temp = pd.concat([df_temp, df_input], axis=0)
            df_temp['SETTLEMENTDATE'] = pd.to_datetime(df_temp['SETTLEMENTDATE'])
            
numfiles = numfiles + 1
outputfile = 'F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchScada_5minDataFrame/Nov16/DispatchScada_5minNov17_' + str(numfiles) + '.csv'
http://localhost:8889/notebooks/IPython%20Notebooks/LEP_Assignment/GeneratorInsight_Stage2_Generator%26Interconnector_5minData%20to%2030min%20Aggregate.ipynb#df_temp.to_csv(outputfile,index=False)

In [3]:
analysisPath = 'F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchScada_5minDataFrame/Nov16/'
count = len([f for f in os.listdir(analysisPath) if os.path.isfile(os.path.join(analysisPath, f))])
print(count)

9


In [4]:
df_final = pd.DataFrame()
for file in os.listdir(analysisPath):
    if file.endswith(".csv"):
        input_file = os.path.join(analysisPath, file)
        # Read the csv files into dataframes
        df_input = pd.read_csv(input_file,sep=',',skiprows=0,error_bad_lines=False,warn_bad_lines=False)
        df_final = pd.concat([df_final, df_input], axis=0)

In [5]:
#Display contents
display(df_final.shape)
display(df_final.head(4))

(2341654, 7)

,I,DISPATCH,UNIT_SCADA,1,SETTLEMENTDATE,DUID,SCADAVALUE
0,D,DISPATCH,UNIT_SCADA,1.0,2016-11-01 00:10:00,BUTLERSG,8.599998
1,D,DISPATCH,UNIT_SCADA,1.0,2016-11-01 00:10:00,CALL_A_4,0.000000
2,D,DISPATCH,UNIT_SCADA,1.0,2016-11-01 00:10:00,CAPTL_WF,0.000000
3,D,DISPATCH,UNIT_SCADA,1.0,2016-11-01 00:10:00,CATHROCK,30.649998


In [6]:
df_final['SETTLEMENTDATE'] = pd.to_datetime(df_final['SETTLEMENTDATE'], errors='coerce')
df_final = df_final.dropna(subset=['SETTLEMENTDATE'])
df_final['SETTLEMENTDATE'] = df_final['SETTLEMENTDATE'] - timedelta(minutes=1)

In [7]:
df_final['DATE'] = df_final['SETTLEMENTDATE'].dt.date
df_final['HOUR'] = df_final['SETTLEMENTDATE'].dt.hour
df_final['MINUTE'] = df_final['SETTLEMENTDATE'].dt.minute
print(df_final['MINUTE'].head(5))

0    9
1    9
2    9
3    9
4    9
Name: MINUTE, dtype: int64


In [8]:
df_final.drop(['I','DISPATCH','UNIT_SCADA','1','SETTLEMENTDATE'], axis=1,inplace=True)

In [9]:
print(df_final.head(5))

       DUID  SCADAVALUE        DATE  HOUR  MINUTE
0  BUTLERSG    8.599998  2016-11-01     0       9
1  CALL_A_4    0.000000  2016-11-01     0       9
2  CAPTL_WF    0.000000  2016-11-01     0       9
3  CATHROCK   30.649998  2016-11-01     0       9
4  CHALLHWF   22.000000  2016-11-01     0       9


In [10]:
mask1 = (df_final['MINUTE'] >  0.0 ) & (df_final['MINUTE'] <=  30.0 )
mask2 = (df_final['MINUTE'] >  30.0 ) & (df_final['MINUTE'] <=  59.0 )

In [11]:
df_Group1 = df_final.loc[mask1]
df_Group2 = df_final.loc[mask2]
print(df_Group1.tail(5))
print(df_Group2.tail(5))

            DUID  SCADAVALUE        DATE  HOUR  MINUTE
176199  YARWUN_1   129.09999  2016-12-01     0       4
176200     YWPS1   379.44809  2016-12-01     0       4
176201     YWPS2   349.87341  2016-12-01     0       4
176202     YWPS3   385.35822  2016-12-01     0       4
176203     YWPS4     0.00000  2016-12-01     0       4
            DUID  SCADAVALUE        DATE  HOUR  MINUTE
175928  YARWUN_1   129.39999  2016-11-30    23      59
175929     YWPS1   379.44809  2016-11-30    23      59
175930     YWPS2   349.37216  2016-11-30    23      59
175931     YWPS3   383.66455  2016-11-30    23      59
175932     YWPS4     0.00000  2016-11-30    23      59


In [12]:
df_30min_Group1 = df_final.loc[mask1].groupby(['DUID','HOUR','DATE'],as_index=False)['SCADAVALUE'].mean()
df_30min_Group1['MINUTE'] = 30.0
print(df_30min_Group1.tail(5))

         DUID  HOUR        DATE  SCADAVALUE  MINUTE
194273  YWPS4    23  2016-11-26         0.0    30.0
194274  YWPS4    23  2016-11-27         0.0    30.0
194275  YWPS4    23  2016-11-28         0.0    30.0
194276  YWPS4    23  2016-11-29         0.0    30.0
194277  YWPS4    23  2016-11-30         0.0    30.0


In [13]:
df_30min_Group2 = df_final.loc[mask2].groupby(['DUID','HOUR','DATE'],as_index=False)['SCADAVALUE'].mean()
df_30min_Group2['MINUTE'] = 59.0
print(df_30min_Group2.tail(5))

         DUID  HOUR        DATE  SCADAVALUE  MINUTE
194006  YWPS4    23  2016-11-26         0.0    59.0
194007  YWPS4    23  2016-11-27         0.0    59.0
194008  YWPS4    23  2016-11-28         0.0    59.0
194009  YWPS4    23  2016-11-29         0.0    59.0
194010  YWPS4    23  2016-11-30         0.0    59.0


In [14]:
df_30min = pd.concat([df_30min_Group1, df_30min_Group2], axis=0)

In [15]:
print(df_30min.tail(5))

         DUID  HOUR        DATE  SCADAVALUE  MINUTE
194006  YWPS4    23  2016-11-26         0.0    59.0
194007  YWPS4    23  2016-11-27         0.0    59.0
194008  YWPS4    23  2016-11-28         0.0    59.0
194009  YWPS4    23  2016-11-29         0.0    59.0
194010  YWPS4    23  2016-11-30         0.0    59.0


In [16]:
print(df_30min['DATE'].describe())

count         388289
unique            31
top       2016-11-26
freq           12960
Name: DATE, dtype: object


In [17]:
df_30min['DATE'] = df_30min['DATE'].astype('datetime64[ns]')

In [18]:
df_30min['DATETIME'] = df_30min['DATE'] + pd.to_timedelta(df_30min['HOUR'], unit='h') + pd.to_timedelta(df_30min['MINUTE'], unit='m')

In [19]:
print(df_30min.tail(5))

         DUID  HOUR       DATE  SCADAVALUE  MINUTE            DATETIME
194006  YWPS4    23 2016-11-26         0.0    59.0 2016-11-26 23:59:00
194007  YWPS4    23 2016-11-27         0.0    59.0 2016-11-27 23:59:00
194008  YWPS4    23 2016-11-28         0.0    59.0 2016-11-28 23:59:00
194009  YWPS4    23 2016-11-29         0.0    59.0 2016-11-29 23:59:00
194010  YWPS4    23 2016-11-30         0.0    59.0 2016-11-30 23:59:00


In [20]:
## Need to add one minute to times with minute = 59
mask = (df_30min['MINUTE']==59.0)
df_30min.loc[mask,'DATETIME']= df_30min.loc[mask,'DATETIME'] + timedelta(minutes=1)

In [21]:
print(df_30min.tail(5))

         DUID  HOUR       DATE  SCADAVALUE  MINUTE   DATETIME
194006  YWPS4    23 2016-11-26         0.0    59.0 2016-11-27
194007  YWPS4    23 2016-11-27         0.0    59.0 2016-11-28
194008  YWPS4    23 2016-11-28         0.0    59.0 2016-11-29
194009  YWPS4    23 2016-11-29         0.0    59.0 2016-11-30
194010  YWPS4    23 2016-11-30         0.0    59.0 2016-12-01


In [22]:
## Need to add one minute to times with minute = 59
mask = (df_30min['MINUTE']==59.0)
df_30min.loc[mask,'MINUTE']= 0.0

In [23]:
print(df_30min.tail(5))

         DUID  HOUR       DATE  SCADAVALUE  MINUTE   DATETIME
194006  YWPS4    23 2016-11-26         0.0     0.0 2016-11-27
194007  YWPS4    23 2016-11-27         0.0     0.0 2016-11-28
194008  YWPS4    23 2016-11-28         0.0     0.0 2016-11-29
194009  YWPS4    23 2016-11-29         0.0     0.0 2016-11-30
194010  YWPS4    23 2016-11-30         0.0     0.0 2016-12-01


In [24]:
df_30min.drop(['DATE','HOUR','MINUTE'], axis=1,inplace=True)

In [25]:
print(df_30min.tail(5))

         DUID  SCADAVALUE   DATETIME
194006  YWPS4         0.0 2016-11-27
194007  YWPS4         0.0 2016-11-28
194008  YWPS4         0.0 2016-11-29
194009  YWPS4         0.0 2016-11-30
194010  YWPS4         0.0 2016-12-01


In [26]:
print(df_30min.head(5))

     DUID  SCADAVALUE            DATETIME
0  AGLHAL    0.000000 2016-11-01 00:30:00
1  AGLHAL    0.000000 2016-11-02 00:30:00
2  AGLHAL    3.364665 2016-11-03 00:30:00
3  AGLHAL    0.000000 2016-11-04 00:30:00
4  AGLHAL    0.000000 2016-11-05 00:30:00


In [27]:
df_output = df_30min.pivot(index='DATETIME', columns='DUID', values='SCADAVALUE')

In [28]:
df_output['DATETIME'] = df_output.index

In [29]:
print(df_output.head(5))

DUID                 AGLHAL  AGLSOM  ANGAST1      ARWF1   BALDHWF1  BARCALDN  \
DATETIME                                                                       
2016-11-01 00:30:00     0.0     0.0      0.0  17.540000  41.548000       0.0   
2016-11-01 01:00:00     0.0     0.0      0.0   8.316667  49.845167       0.0   
2016-11-01 01:30:00     0.0     0.0      0.0   8.783333  59.467667       0.0   
2016-11-01 02:00:00     0.0     0.0      0.0  10.416667  74.911500       0.0   
2016-11-01 02:30:00     0.0     0.0      0.0   8.516667  72.100167       0.0   

DUID                 BARRON-1  BARRON-2    BASTYAN  BBTHREE1  \
DATETIME                                                       
2016-11-01 00:30:00       0.0       0.0  78.992006       0.0   
2016-11-01 01:00:00       0.0       0.0  79.018338       0.0   
2016-11-01 01:30:00       0.0       0.0  79.021672       0.0   
2016-11-01 02:00:00       0.0       0.0  79.165007       0.0   
2016-11-01 02:30:00       0.0       0.0  79.253340     

In [30]:
print(df_output.shape)

(1441, 271)


In [31]:
outputfile='F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchScada_Nov16_30minAverage.csv'
df_output.to_csv(outputfile,index=False)